In [77]:
import json

In [89]:
vocabs_pool: set[str] = set()
exclusive_pool: set[str] = set()
alphabets: set[str] = set()

with open(r"vocabs\ja\ja-len4-easy.json") as file:
    json_data = json.load(file)
    for d in json_data:
        vocabs_pool.add(d)
        exclusive_pool.add(d)
        alphabets = alphabets.union(set(d))

# with open(r"vocabs\en\en-len4-medium.json") as file:
#     json_data = json.load(file)
#     for d in json_data:
#         exclusive_pool.remove(d)

num_vocabs = len(vocabs_pool)
print(len(alphabets), "".join(sorted(alphabets)))
print(f"{len(vocabs_pool)=}")
print(f"{len(exclusive_pool)=}")

81 ァアィイウェエォオカガキギクグケゲコゴサザシジスズセゼソゾタダチヂッツヅテデトドナニヌネノハバパヒビピフブプヘベペホボポマミムメモャヤュユョヨラリルレロワヲンヴー
len(vocabs_pool)=13155
len(exclusive_pool)=13155


In [79]:
def change_character(vocab: str) -> set[str]:
    new_vocabs = set()
    for i, c in enumerate(vocab):
        for a in alphabets:
            if a != c:
                new_vocab = vocab[:i] + a + vocab[i + 1 :]
                new_vocabs.add(new_vocab)

    return new_vocabs


print(change_character("abc"))

{'ヒbc', 'ィbc', 'abュ', 'キbc', 'aザc', 'aカc', 'abー', 'ドbc', 'abザ', 'abム', 'aギc', 'ゲbc', 'ラbc', 'デbc', 'aッc', 'セbc', 'abサ', 'ョbc', 'abギ', 'ニbc', 'モbc', 'aェc', 'アbc', 'abケ', 'エbc', 'abソ', 'フbc', 'ブbc', 'abユ', 'abハ', 'abノ', 'オbc', 'aトc', 'aャc', 'abゲ', 'ンbc', 'ァbc', 'aクc', 'abボ', 'aゲc', 'aルc', 'aィc', 'ズbc', 'aガc', 'マbc', 'aオc', 'aツc', 'aゼc', 'abヤ', 'abウ', 'ヘbc', 'チbc', 'abカ', 'abポ', 'abヒ', 'ェbc', 'aヨc', 'aヅc', 'aニc', 'スbc', 'タbc', 'aゾc', 'ヤbc', 'aュc', 'abリ', 'ゴbc', 'ノbc', 'aブc', 'aホc', 'aウc', 'abモ', 'abダ', 'ゼbc', 'ミbc', 'abヨ', 'abネ', 'ザbc', 'abジ', 'abズ', 'abゾ', 'aモc', 'aヒc', 'ュbc', 'aアc', 'abゼ', 'aノc', 'abイ', 'ケbc', 'abン', 'ボbc', 'aエc', 'aデc', 'トbc', 'aメc', 'aヘc', 'abフ', 'ーbc', 'abツ', 'コbc', 'aコc', 'ハbc', 'aテc', 'ウbc', 'abバ', 'abプ', 'ギbc', 'ルbc', 'abメ', 'aグc', 'aボc', 'aビc', 'ムbc', 'aスc', 'aシc', 'abマ', 'abパ', 'ポbc', 'abエ', 'aゴc', 'aイc', 'aネc', 'abァ', 'バbc', 'aケc', 'aラc', 'aベc', 'abト', 'aソc', 'abド', 'ダbc', 'ガbc', 'aァc', 'abィ', 'ホbc', 'abヅ', 'カbc', 'aタc', 'abタ', 'aズc', 'ロbc', 'aョc', 'aフc', 'シbc'

In [80]:
def character_difference(from_: str, to: str) -> int:
    count = 0
    for c1, c2 in zip(from_, to):
        if c1 != c2:
            count += 1
    return count

In [81]:
def distance(from_: str, to: str, max_distance=None) -> tuple[int, list[str]]:
    vocab_path = {from_: ""}
    queue = [(from_, 0)]
    while queue:
        (cur_vocab, cur_distance) = queue.pop(0)
        if cur_vocab == to:
            path = [cur_vocab]
            while path[0] != from_:
                path.insert(0, vocab_path[path[0]])
            return (cur_distance, path)

        if cur_distance == max_distance:
            continue

        for new_vocab in change_character(cur_vocab):
            if (new_vocab not in vocab_path) and (new_vocab in vocabs_pool):
                vocab_path[new_vocab] = cur_vocab
                queue.append((new_vocab, cur_distance + 1))

    return (-1, [])

In [82]:
print(distance("admin", "admit"))
print(distance("dicho", "hecha"))
print(distance("war", "him"))

(-1, [])
(-1, [])
(-1, [])


In [83]:
max_distance = 4
strict = False
count = 0
count_exclusive = 0
starter = set()
involved = set()
pairs: list[tuple[str, str]] = []

for i, v1 in enumerate(vocabs_pool):
    for j, v2 in enumerate(vocabs_pool):
        dist = distance(v1, v2, max_distance=max_distance)
        diff = character_difference(v1, v2)
        if (strict and dist[0] > diff) or (not strict and dist[0] > 3):
            count += 1
            starter.add(v1)
            involved = involved.union(set(dist[1]))
            # print(dist)

            if any(v in exclusive_pool for v in dist[1]):
                count_exclusive += 1
                pairs.append((v1, v2))

    if i % (num_vocabs // 10) == 0:
        print(f'checking "{v1}"| {i}/{num_vocabs}')

total_combinations = num_vocabs * (num_vocabs - 1)

pair_rate = count / total_combinations
print(f"pair rate: {count}/{total_combinations} = {pair_rate:.2%}")

exclusive_rate = count_exclusive / total_combinations
print(
    f"exclusive pair rate: {count_exclusive}/{total_combinations} = {exclusive_rate:.2%}"
)

starter_rate = len(starter) / num_vocabs
print(f"starter: {len(starter)}/{num_vocabs} = {starter_rate:.2%}")

participation_rate = len(involved) / num_vocabs
print(f"participation: {len(involved)}/{num_vocabs} = {participation_rate:.2%}")

checking "シリョウ"| 0/607
checking "ゲンパツ"| 60/607
checking "キコエル"| 120/607
checking "ヤサシイ"| 180/607
checking "ソノモノ"| 240/607
checking "タショウ"| 300/607
checking "コウカイ"| 360/607
checking "タタカウ"| 420/607
checking "タイセイ"| 480/607
checking "ドウシテ"| 540/607
checking "イッパン"| 600/607
pair rate: 2254/367842 = 0.61%
exclusive pair rate: 2254/367842 = 0.61%
starter: 224/607 = 36.90%
participation: 244/607 = 40.20%


In [86]:
with open(rf"vocabs\ja\ja-len4-pairs-easy.json", "w") as f:
    json.dump(pairs, f)

In [85]:
def create_distance_graph():
    distance_graph: dict[tuple[str, str], tuple[int, list[str]]] = {}

    def calculate_distance(from_: str, to: str):
        if (from_, to) in distance_graph:
            return

        min_distance = -1
        min_path = []
        for new_vocab in change_character(from_):
            if new_vocab in vocabs_pool:
                calculate_distance(new_vocab, to)
                distance, path = distance_graph[(new_vocab, to)]
                if min_distance == -1 or 0 <= distance < min_distance:
                    min_distance = distance + 1
                    min_path = [from_] + path
        distance_graph[(from_, to)] = (min_distance, min_path)

    for v1 in vocabs_pool:
        for v2 in vocabs_pool:
            calculate_distance(v1, v2)
    return distance_graph


# distance_graph = create_distance_graph()
# print(distance_graph)